In [0]:
from pyspark.sql.functions import expr
catalog_name = "healthcare_catalog" 
schema_name = 'raw_zone'
docs_path = '/Volumes/healthcare_catalog/raw_zone/encounter_pdfs'
docs_df = spark.read.format("binaryFile").option("pathGlobFilter", "*.pdf").option("recursiveFileLookup", "true").load(docs_path)
parsed_df = docs_df.withColumn("parsed_content", expr(f"""ai_parse_document(content, map("version", "2.0"))"""))
parsed_df = parsed_df.drop("content")
output_table = f"{catalog_name}.{schema_name}.docs_parsed"
parsed_df.write.format("delta").mode("overwrite").saveAsTable(output_table)
print(f"✅ Parsed results saved to Delta table: {output_table}")

In [0]:

%sql
SELECT * FROM healthcare_catalog.raw_zone.docs_parsed

In [0]:
%skip
%sql
WITH parsed AS (
  SELECT
    path,
    from_json(
      CAST(parsed_content AS STRING),
      'STRUCT<
         document: STRUCT<
           elements: ARRAY<
             STRUCT<
               bbox: ARRAY<STRUCT<coord: ARRAY<INT>, page_id: INT>>,
               content: STRING,
               description: STRING,
               id: INT,
               type: STRING
             >
           >
         >
      >'
    ) AS pc
  FROM healthcare_catalog.raw_zone.docs_parsed
),exploded AS (
  SELECT
    path,
    element.bbox[0].page_id AS page_id,
    element.content AS content
  FROM parsed
  LATERAL VIEW explode(pc.document.elements) AS element
),extracted as (

SELECT
  path,
  page_id,
  collect_list(content) AS comments
FROM exploded
GROUP BY path, page_id
ORDER BY page_id
)

SELECT 
page_id,
ai_extract(array_join(comments, ' '), array('MRN')).`MRN` AS MRN,
ai_extract(array_join(comments, ' '), array('PATIENT')).`PATIENT` AS PATIENT,
ai_extract(array_join(comments, ' '), array('ENCOUNTER DATE')).`ENCOUNTER DATE` AS ENCOUNTER_DATE,
comments
FROM extracted

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# -----------------------------
# 1. Define the schema used in from_json
# -----------------------------
schema = StructType([
    StructField(
        "document",
        StructType([
            StructField(
                "elements",
                ArrayType(
                    StructType([
                        StructField(
                            "bbox",
                            ArrayType(
                                StructType([
                                    StructField("coord", ArrayType(IntegerType())),
                                    StructField("page_id", IntegerType())
                                ])
                            )
                        ),
                        StructField("content", StringType()),
                        StructField("description", StringType()),
                        StructField("id", IntegerType()),
                        StructField("type", StringType())
                    ])
                )
            )
        ])
    )
])

# -----------------------------
# 2. parsed CTE
# -----------------------------
parsed_df = (
    spark.table("healthcare_catalog.raw_zone.docs_parsed")
        .select(
            "path",
            F.from_json(
                F.col("parsed_content").cast("string"),
                schema
            ).alias("pc")
        )
)

# -----------------------------
# 3. exploded CTE
# -----------------------------
exploded_df = (
    parsed_df
        .withColumn("element", F.explode("pc.document.elements"))
        .select(
            "path",
            F.col("element.bbox")[0]["page_id"].alias("page_id"),
            F.col("element.content").alias("content")
        )
)

# -----------------------------
# 4. extracted CTE
# -----------------------------
extracted_df = (
    exploded_df
        .groupBy("path", "page_id")
        .agg(
            F.collect_list("content").alias("Notes")
        )
)

# -----------------------------
# 5. Final select with ai_extract
# -----------------------------
final_df = (
    extracted_df
        .select(
            #"page_id",
            F.expr("ai_extract(array_join(Notes, ' '), array('MRN')).MRN").alias("MRN"),
            F.expr("ai_extract(array_join(Notes, ' '), array('PATIENT')).PATIENT").alias("PATIENT"),
            F.expr("ai_extract(array_join(Notes, ' '), array('ENCOUNTER DATE')).`ENCOUNTER DATE`")
              .alias("ENCOUNTER_DATE"),
            "Notes"
        )
        .orderBy("page_id")
)

# Display or write
display(final_df)

In [0]:
output_table_silver = "healthcare_catalog.silver_zone.extracted_entities1"

spark.sql(f"DROP TABLE IF EXISTS {output_table_silver}")

final_df.write.format("delta").mode("overwrite").saveAsTable(output_table_silver)
print(f"✅ final_df results saved to Delta table: {output_table_silver}")

In [0]:
%sql
SELECT * FROM healthcare_catalog.silver_zone.extracted_entities

In [0]:
output_table_silver = "healthcare_catalog.silver_zone.extracted_entities"

spark.sql(f"DROP TABLE IF EXISTS {output_table_silver}")

final_df.write.format("delta").mode("overwrite").saveAsTable(output_table_silver)
#spark.sql(f"GRANT ALL PRIVILEGES ON TABLE {output_table_silver} TO `everyone`")
print(f"✅ Parsed results saved to Delta table: {output_table_silver} and granted ALL privileges to everyone")

In [0]:
%sql
SELECT * FROM healthcare_catalog.silver_zone.extracted_entities 

In [0]:
%sql
WITH RW AS (
  SELECT *, explode(regexp_extract_all(CAST(parsed_content AS STRING), 'bbox"\s*:\s*"([^"]*)')) AS comments
FROM healthcare_catalog.raw_zone.docs_parsed)
SELECT
ai_extract(comments, array('page_id')).`page_id` AS page_id, 
ai_extract(comments, array('ENCOUNTER DATE')).`ENCOUNTER DATE` AS encounter_date,
ai_extract(comments, array('PATIENT')).`PATIENT` AS patient_name,
ai_extract(comments, array('MRN')).`MRN` AS MRN,
ai_extract(comments, array('CHIEF COMPLAINT:')).`CHIEF COMPLAINT:` AS CHIEF_COMPLAINT,
ai_extract(comments, array('OBJECTIVE:')).`OBJECTIVE:` AS OBJECTIVE,
ai_extract(comments, array('IMPRESSION:')).`IMPRESSION:` AS IMPRESSION,
ai_extract(comments, array('REASON FOR REFERRAL:')).`REASON FOR REFERRAL:` AS REASON_FOR_REFERRAL
FROM RW




In [0]:
%sql
SELECT
  path,
  modificationTime,
  length,
  parsed_content:document:pages,
  parsed_content:document:elements,
  parsed_content:error_status,
  parsed_content:metadata
FROM healthcare_catalog.raw_zone.docs_parsed

In [0]:
%sql
SELECT * FROM healthcare_catalog.raw_zone.parsed_documents

In [0]:
%sql
WITH elements AS (
  SELECT
    path,
    modificationTime,
    element.bbox[0].page_id AS page_id,
    element.content
  FROM healthcare_catalog.raw_zone.docs_parsed
  LATERAL VIEW explode(parsed_content:document:elements) AS element
),
extracted AS (
  SELECT
    path,
    modificationTime,
    page_id,
    ai_extract(content, array(
      'ENCOUNTER DATE',
      'PATIENT',
      'MRN',
      'CHIEF COMPLAINT:',
      'SUBJECTIVE:',
      'OBJECTIVE:',
      'ASSESSMENT:',
      'PLAN:'
    )) AS extracted_fields
  FROM elements
)
SELECT
  path,
  modificationTime,
  page_id,
  extracted_fields.`ENCOUNTER DATE` AS encounter_date,
  extracted_fields.`PATIENT` AS patient_name,
  extracted_fields.`MRN` AS MRN,
  extracted_fields.`CHIEF COMPLAINT:` AS chief_complaint,
  extracted_fields.`SUBJECTIVE:` AS subjective,
  extracted_fields.`OBJECTIVE:` AS objective,
  extracted_fields.`ASSESSMENT:` AS assessment,
  extracted_fields.`PLAN:` AS plan
FROM extracted
GROUP BY path, modificationTime, page_id, encounter_date, patient_name, MRN, chief_complaint, subjective, objective, assessment, plan
ORDER BY path, page_id;